# README

Text

# Set Paths

In [2]:
import os

# Configurable values
dir_location = 'internal' # Location that this run is on: internal, external, or cloud (i.e. OneDrive)
output_to_cloud = False # Overrides the output path to always output to the cloud (i.e. OneDrive)
is_original_data = True # Original data is the dataset used by the authors of ParticleSeg3D, False results in the DigiM tablet dataset
weights_tag = 'original_particle_seg' # Name of the weights that will be used
run_tag = 'original_dataset' # Name of the run, like what name will be used to label the outputs generated from this run

# Set path to the base directory
if dir_location.lower() == 'internal':
    base_path = r'C:\Senior_Design'
elif dir_location.lower() == 'external':
    base_path = r'D:\Senior_Design'
elif dir_location.lower() == 'cloud':
    base_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Files'

# Set base paths to input, output, and weights        
base_input_path = os.join.path(base_path, 'database')
base_output_path = os.join.path(base_path, 'outputs')
if output_to_cloud:
    base_output_path = os.join.path(r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Files', 'outputs')
base_weights_path = os.join.path(base_path, 'weights')

# Set Zarr (model output) and TIFF (readable output) paths
output_zarr_path = os.join.path(base_output_path, 'zarr', run_tag)
output_tiff_path = os.join.path(base_output_path, 'tiff', run_tag)

# Set dataset path
if is_original_data:
    input_path = os.join.path(base_input_path, 'orignal_dataset')
else:
    input_path = os.join.path(base_input_path, 'tablet_dataset')

# Set weights path
weights_path = os.join.path(base_weights_path, weights_tag)

# Ensure paths with spaces or parentheses are quoted (for using the directory in OneDrive, does not matter if there are no spaces or parentheses)
input_path = f'"{input_path}"'
output_zarr_path = f'"{output_zarr_path}"'
output_tiff_path = f'"{output_tiff_path}"'
weights_path = f'"{weights_path}"'

# Check CUDA Availability

In [3]:
import torch
torch.cuda.is_available()

True

# Run ParticleSeg3D Using Subprocess Module

In [ ]:
import subprocess

# Inference
subprocess.run(['ps3d_inference', '-i', input_path, '-o', output_zarr_path, '-m', weights_path], capture_output=True) # ps3d_inference -i /path/to/input -o /path/to/output -m /path/to/weights

# Define the commands to execute in Anaconda PowerShell Prompt
activate_env = 'conda activate Senior_Design_py310'
inference = 'ps3d_inference -i ' + input_path + ' -o ' + output_zarr_path + ' -m ' + weights_path
command = activate_env + '; ' + inference

# Run the commands
process = subprocess.run(
    ['powershell', '-Command', command], 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE, 
    text=True
)

# Output the results
print("STDOUT:", process.stdout)
print("STDERR:", process.stderr)